In [39]:
import math
import numpy as np
from PIL import Image
from math import log10, sqrt
import os
import cv2
import torch
import matplotlib.pyplot as plt
# Path to the directory
path = "experiments/cosine/test//results/"
image_folder = path
def img2array(image_folder):
    # Get all image file names in the folder
    image_files = os.listdir(image_folder)
    # List to store all image tensors
    images = []
    for image_file in image_files:
        # Build complete image path
        image_path = os.path.join(image_folder, image_file)
        
        # Read the image using OpenCV
        img = cv2.imread(image_path)
        
        # Convert the image from BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Convert the numpy array to torch tensor and add channel dimension
        img_tensor = img.transpose(2, 0, 1)
        
        # Add the image tensor to the list
        images.append(img_tensor)

    # Stack all image tensors together along a new dimension
    images_tensor = np.stack(images)
    return images_tensor

In [40]:
#images_tensor = img2array(image_folder)
#print(images_tensor.shape)
#lt.imshow(images_tensor[0].permute(1, 2, 0))

In [41]:

# Open the image form working directory
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))
def PSNR2(original, compressed):
    original = original.astype(np.float64)
    compressed= compressed.astype(np.float64)
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [42]:

#i1  = np.asarray(Image.open(path + "hr"))
#i2  = np.asarray(Image.open(path + "sr"))
i1 = img2array(path + "hr")
i2 = img2array(path + "sr")
print(i1.max(),i1.min(),type(i1[0,0,0]))
print(calculate_psnr(i1,i2))
print(PSNR2(i1,i2))

255 0 <class 'numpy.ndarray'>
22.783997781397723
22.783997781397723


In [43]:
import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [44]:
print(calculate_ssim(i1[1],i2[1]))

None
